### 3.6. softmax回归的从零开始实现

就像我们从零开始实现线性回归一样， 我们认为softmax回归也是重要的基础，因此你应该知道实现softmax回归的细节。 本节我们将使用刚刚在 3.5节中引入的Fashion-MNIST数据集， 并设置数据迭代器的批量大小为256。

In [1]:
import torch
from IPython import display
from d2l import torch as d2l

batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size)

/home/mkid/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811748419/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


#### 3.6.1. 初始化模型参数

和之前线性回归的例子一样，这里的每个样本都将用固定长度的向量表示。 原始数据集中的每个样本都是28x28的图像。 在本节中，我们将展平每个图像，把它们看作长度为784的向量。 在后面的章节中，我们将讨论能够利用图像空间结构的特征， 但现在我们暂时只把每个像素位置看作一个特征。

回想一下，在softmax回归中，我们的输出与类别一样多。 因为我们的数据集有10个类别，所以网络输出维度为10。 因此，权重将构成一个784x10的矩阵， 偏置将构成一个1x10的行向量。 与线性回归一样，我们将使用正态分布初始化我们的权重W，偏置初始化为0。

In [ ]:
num_inputs = 784
num_outputs = 10

W = torch.normal(0, 0.01, size=(num_inputs, num_outputs), requires_grad=True)
b = torch.zeros(num_outputs, requires_grad=True)

#### 3.6.2. 定义softmax操作

In [2]:
X = torch.tensor([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
X.sum(0, keepdim=True), X.sum(1, keepdim=True)

(tensor([[5., 7., 9.]]),
 tensor([[ 6.],
         [15.]]))

In [11]:
def softmax(X):
    X_exp = torch.exp(X)
    partition = X_exp.sum(1, keepdim=True)
    # print(f'partition = {partition}')
    return X_exp / partition  # 这里应用了广播机制

In [12]:
X = torch.normal(0, 1, (2, 5))
# print(f'X = {X}, {torch.exp(X).sum(1)}')
X_prob = softmax(X)
X_prob, X_prob.sum(1)

(tensor([[0.0261, 0.2655, 0.1569, 0.2838, 0.2677],
         [0.0713, 0.3648, 0.3401, 0.0570, 0.1669]]),
 tensor([1., 1.]))

#### 3.6.3. 定义模型

In [14]:
def net(X):
    return softmax(torch.matmul(X.reshape((-1, W.shape[0])), W) + b)    # y = softmax(WX + b)

#### 3.6.4. 定义损失函数

接下来，我们实现 3.4节中引入的交叉熵损失函数。 这可能是深度学习中最常见的损失函数，因为目前分类问题的数量远远超过回归问题的数量。

交叉熵采用真实标签的预测概率的负对数似然。


我们创建一个数据样本y_hat，其中包含2个样本在3个类别的预测概率， 以及它们对应的标签y。 有了y，我们知道在第一个样本中，第一类是正确的预测； 而在第二个样本中，第三类是正确的预测。 然后使用y作为y_hat中概率的索引， 我们选择第一个样本中第一个类的概率和第二个样本中第三个类的概率。

In [20]:
y = torch.tensor([0, 2])
y_hat = torch.tensor([[0.1, 0.3, 0.6], [0.3, 0.2, 0.5]])
y_hat[[0, 1], y]
# 测试y_hat.type(y.dtype) == y的用法
y_hat.type()

'torch.FloatTensor'

In [17]:
def cross_entropy(y_hat, y):
    return - torch.log(y_hat[range(len(y_hat)), y]) # y_hat = net(X)，而y是真实标签的索引，y_hat[y_hat[0:len], y]返回的是对每个x的预测概率

cross_entropy(y_hat, y)

tensor([2.3026, 0.6931])

#### 3.6.5. 分类精度

In [18]:
def accuracy(y_hat, y):  #@save
    """计算预测正确的数量"""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return float(cmp.type(y.dtype).sum())